In [1]:
from sys import modules

IN_COLAB = 'google.colab' in modules
if IN_COLAB:
    !pip install -q ir_axioms python-terrier

In [2]:
# Start/initialize PyTerrier.
from pyterrier import started, init

if not started():
    init(tqdm="notebook")

PyTerrier 0.8.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [3]:
from pathlib import Path
from typing import List
from pyterrier import Transformer
from examples.trec_util import TrecTrack

# track = TrecTrack(28, "deep", "msmarco-passage/trec-dl-2019")
track = TrecTrack(
    28, "deep", "msmarco-passage/trec-dl-2019",
    base_results_dir=Path("/home/heinrich/Development/webis/trec-system-runs")
)
retrieval_systems: List[Transformer] = [result % 10 for result in track.results]

Read results: 100%|██████████| 37/37 [00:13<00:00,  2.66run/s]


In [4]:
from ir_axioms.backend.pyterrier.experiment import AxiomaticExperiment
from ir_axioms.axiom import TFC1, LNC1

axiomatic_experiment = AxiomaticExperiment(
    retrieval_systems=retrieval_systems,
    topics=track.dataset.get_topics(),
    qrels=track.dataset.get_qrels(),
    index=track.index,
    dataset=track.ir_dataset,
    axioms=[TFC1(), LNC1()],
    filter_by_qrels=True,
    filter_by_topics=True,
    verbose=True,
)
axiomatic_experiment

AxiomaticExperiment(retrieval_systems=[RankCutoffTransformer(ComposedPipeline(SourceTransformer(), TaggedTransformer(tag='/home/heinrich/Development/webis/trec-system-runs/trec28/deep/input.srchvrs_ps_run1.gz')), Scalar('10')), RankCutoffTransformer(ComposedPipeline(SourceTransformer(), TaggedTransformer(tag='/home/heinrich/Development/webis/trec-system-runs/trec28/deep/input.ICT-BERT2.gz')), Scalar('10')), RankCutoffTransformer(ComposedPipeline(SourceTransformer(), TaggedTransformer(tag='/home/heinrich/Development/webis/trec-system-runs/trec28/deep/input.TUW19-p1-f.gz')), Scalar('10')), RankCutoffTransformer(ComposedPipeline(SourceTransformer(), TaggedTransformer(tag='/home/heinrich/Development/webis/trec-system-runs/trec28/deep/input.bm25tuned_rm3_p.gz')), Scalar('10')), RankCutoffTransformer(ComposedPipeline(SourceTransformer(), TaggedTransformer(tag='/home/heinrich/Development/webis/trec-system-runs/trec28/deep/input.bm25tuned_p.gz')), Scalar('10')), RankCutoffTransformer(ComposedP

In [5]:
preferences = axiomatic_experiment.preferences
preferences

AxiomaticExperiment: 100%|██████████| 37/37 [00:05<00:00,  6.69system/s]


,qid,docno_a,rank_a,score_a,name,query,docno_b,rank_b,score_b,original_preference,oracle_preference,TFC1_preference,LNC1_preference
0,156493,2259183,1,6.572989,srchvrs_ps_run1,do goldfish grow,2259183,1,6.572989,0,0,0,0
1,156493,2259183,1,6.572989,srchvrs_ps_run1,do goldfish grow,3288597,2,6.525999,1,0,0,0
2,156493,2259183,1,6.572989,srchvrs_ps_run1,do goldfish grow,3288601,3,6.517504,1,0,0,0
3,156493,2259183,1,6.572989,srchvrs_ps_run1,do goldfish grow,3288596,4,6.508273,1,0,0,-1
4,156493,2259183,1,6.572989,srchvrs_ps_run1,do goldfish grow,3288600,5,6.508273,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3422,146187,1230566,9,0.427014,runid2,difference between a mcdouble and a double che...,6430258,5,0.430223,-1,0,-1,0
3423,146187,1230566,9,0.427014,runid2,difference between a mcdouble and a double che...,6658185,6,0.428952,-1,1,0,0
3424,146187,1230566,9,0.427014,runid2,difference between a mcdouble and a double che...,7096817,7,0.427588,-1,1,0,0
3425,146187,1230566,9,0.427014,runid2,difference between a mcdouble and a double che...,1138801,8,0.427014,-1,1,0,0


In [6]:
# Judged preferences (should be equal to preferences if experiment was alreay filtered by qrels.)
judged_preferences = preferences[preferences["oracle_preference"].notna()]
judged_preferences

,qid,docno_a,rank_a,score_a,name,query,docno_b,rank_b,score_b,original_preference,oracle_preference,TFC1_preference,LNC1_preference
0,156493,2259183,1,6.572989,srchvrs_ps_run1,do goldfish grow,2259183,1,6.572989,0,0,0,0
1,156493,2259183,1,6.572989,srchvrs_ps_run1,do goldfish grow,3288597,2,6.525999,1,0,0,0
2,156493,2259183,1,6.572989,srchvrs_ps_run1,do goldfish grow,3288601,3,6.517504,1,0,0,0
3,156493,2259183,1,6.572989,srchvrs_ps_run1,do goldfish grow,3288596,4,6.508273,1,0,0,-1
4,156493,2259183,1,6.572989,srchvrs_ps_run1,do goldfish grow,3288600,5,6.508273,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3422,146187,1230566,9,0.427014,runid2,difference between a mcdouble and a double che...,6430258,5,0.430223,-1,0,-1,0
3423,146187,1230566,9,0.427014,runid2,difference between a mcdouble and a double che...,6658185,6,0.428952,-1,1,0,0
3424,146187,1230566,9,0.427014,runid2,difference between a mcdouble and a double che...,7096817,7,0.427588,-1,1,0,0
3425,146187,1230566,9,0.427014,runid2,difference between a mcdouble and a double che...,1138801,8,0.427014,-1,1,0,0


In [7]:
wrong_preferences = judged_preferences[
    abs(judged_preferences["oracle_preference"] - judged_preferences["original_preference"]) >= 2
    ]
wrong_preferences

,qid,docno_a,rank_a,score_a,name,query,docno_b,rank_b,score_b,original_preference,oracle_preference,TFC1_preference,LNC1_preference
8,156493,2259183,1,6.572989,srchvrs_ps_run1,do goldfish grow,6139386,9,6.399120,1,-1,0,0
17,156493,3288597,2,6.525999,srchvrs_ps_run1,do goldfish grow,6139386,9,6.399120,1,-1,0,0
26,156493,3288601,3,6.517504,srchvrs_ps_run1,do goldfish grow,6139386,9,6.399120,1,-1,0,0
35,156493,3288596,4,6.508273,srchvrs_ps_run1,do goldfish grow,6139386,9,6.399120,1,-1,0,0
44,156493,3288600,5,6.508273,srchvrs_ps_run1,do goldfish grow,6139386,9,6.399120,1,-1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3420,146187,1230566,9,0.427014,runid2,difference between a mcdouble and a double che...,2040185,3,0.430896,-1,1,0,0
3421,146187,1230566,9,0.427014,runid2,difference between a mcdouble and a double che...,2257806,4,0.430273,-1,1,0,0
3423,146187,1230566,9,0.427014,runid2,difference between a mcdouble and a double che...,6658185,6,0.428952,-1,1,0,0
3424,146187,1230566,9,0.427014,runid2,difference between a mcdouble and a double che...,7096817,7,0.427588,-1,1,0,0


In [8]:
wrong_preferences_with_axiom_hints = wrong_preferences[
    (wrong_preferences["TFC1_preference"] != 0) | (wrong_preferences["LNC1_preference"] != 0)
    ]
wrong_preferences_with_axiom_hints

,qid,docno_a,rank_a,score_a,name,query,docno_b,rank_b,score_b,original_preference,oracle_preference,TFC1_preference,LNC1_preference
94,1110199,398442,2,7.017928,srchvrs_ps_run1,what is wifi vs bluetooth,5218014,5,6.803596,1,-1,1,0
112,1110199,554521,4,6.814814,srchvrs_ps_run1,what is wifi vs bluetooth,5218014,5,6.803596,1,-1,0,-1
118,1110199,5218014,5,6.803596,srchvrs_ps_run1,what is wifi vs bluetooth,398442,2,7.017928,-1,1,-1,0
120,1110199,5218014,5,6.803596,srchvrs_ps_run1,what is wifi vs bluetooth,554521,4,6.814814,-1,1,0,1
246,130510,7125239,1,6.547254,srchvrs_ps_run1,definition declaratory judgment,1494936,4,6.341850,1,-1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3140,182539,8757181,5,0.432632,runid2,example of monotonic function,57444,2,0.434976,-1,1,1,0
3187,1113437,4981632,1,0.429929,runid2,what is physical description of spruce,8128794,4,0.427766,1,-1,-1,0
3206,1113437,6021898,3,0.428806,runid2,what is physical description of spruce,1066532,5,0.427582,1,-1,1,0
3211,1113437,8128794,4,0.427766,runid2,what is physical description of spruce,4981632,1,0.429929,-1,1,1,0


In [9]:
wrong_preferences_per_query = wrong_preferences.groupby(["qid"]).aggregate(wrong_preferences=("original_preference", "count"))
wrong_preferences_per_query.sort_values(by="wrong_preferences", ascending=False, inplace=True)
wrong_preferences_per_query

,wrong_preferences
qid,
47923,1040
405717,1006
207786,874
451602,840
359349,820
148538,818
490595,784
1113437,748
1129237,746


In [10]:
wrong_preferences_per_run = wrong_preferences.groupby(["name"]).aggregate(wrong_preferences=("original_preference", "count"))
wrong_preferences_per_run.sort_values(by="wrong_preferences", ascending=False, inplace=True)
wrong_preferences_per_run

,wrong_preferences
name,
srchvrs_ps_run1,792
TUW19-p3-f,770
TUW19-p1-f,748
TUW19-p2-f,744
TUW19-p3-re,726
TUW19-p2-re,726
TUW19-p1-re,706
UNH_bm25,706
srchvrs_ps_run3,702
